In [1]:
import tensorflow as tf
import pandas as pd
import json
import configparser
from pathlib import Path
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

configs = configparser.ConfigParser()
configs.read("configuracoes.conf",encoding='utf-8')
data_folder = Path(configs["DEFAULT"]["folder_path"])
path = data_folder / configs["DEFAULT"]["file"]
objeto = configs["DEFAULT"]["objeto"]
regiao = configs["DEFAULT"]["regiao"]

look_back = 12 #Quantidade de meses anteriores usado para prever
forecast_horizon = int(configs["DEFAULT"]["meses"]) #Quantidade de meses previstos

# Prever 3 meses

In [2]:
#Carrega os dados
cesta = pd.read_excel(path)
cesta.drop(['ano', 'salario', 'razao'], axis=1, inplace=True)
cesta = cesta / 1000
cesta = cesta.iloc[:-3:, ::]

#Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
for n_step in range(1, look_back + forecast_horizon):
    cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
cesta.dropna(inplace=True)
cesta.reset_index(drop=True, inplace=True)

#Divide os dados em Treino e Teste
X_train = cesta.iloc[::, :look_back:].values
y_train = cesta.iloc[::, look_back::].values
X_val = cesta.iloc[-1::, -12::].values

#remodela os dados para formato (batches, sequencia de tempo, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

#Cria o modelo
model=Sequential()
model.add(LSTM(32, stateful=True, input_shape=(look_back, 1), return_sequences=True, batch_input_shape=(1, look_back, 1)))
model.add(LSTM(32,stateful=True))
model.add(Dense(forecast_horizon))
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

#Treina e prever (forecast_horizon) quantidade de tempo
history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
forecast = model.predict(X_val, batch_size=1)

resultados = [valor for valor in forecast[0]]
objeto = objeto.replace(" ",'_')
output = {objeto.lower():str(resultados)}
filename = f"previsao_LSTM_{objeto.lower()}_{regiao.lower()}_{forecast_horizon}_meses.json"
pasta_atual = os.getcwd()
full_url = Path(pasta_atual) / 'resultados' / 'previsoes' / filename
os.makedirs(os.path.dirname(full_url), exist_ok=True)
with open(full_url,"w") as file:
    json.dump(output,file,ensure_ascii=False)
print("Arquivo com resultados gerados")

Epoch 1/150
187/187 [==============================] - 1s 3ms/step - loss: 0.0021
Epoch 2/150
187/187 [==============================] - 1s 3ms/step - loss: 0.0015
Epoch 3/150
187/187 [==============================] - 1s 3ms/step - loss: 0.0018
Epoch 4/150
187/187 [==============================] - 1s 3ms/step - loss: 0.0018
Epoch 5/150
187/187 [==============================] - 1s 4ms/step - loss: 0.0018
Epoch 6/150
187/187 [==============================] - 1s 3ms/step - loss: 0.0018
Epoch 7/150
187/187 [==============================] - 1s 3ms/step - loss: 0.0016
Epoch 8/150
187/187 [==============================] - 1s 3ms/step - loss: 0.0017
Epoch 9/150
187/187 [==============================] - 1s 3ms/step - loss: 0.0016
Epoch 10/150
187/187 [==============================] - 1s 3ms/step - loss: 0.0014
Epoch 11/150
187/187 [==============================] - 1s 3ms/step - loss: 0.0013
Epoch 12/150
187/187 [==============================] - 1s 4ms/step - loss: 0.0012A: 0s - loss:
